In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from mcmc_optimization import optimize_via_mcmc
from models import LinearLayer
from data_samplers import BatchSampler
from proposal_generators import (
    LinearLayerUnivariateNormalProposalGenerator,
    LinearLayerMultivariateNormalProposalGenerator,
)